# Classificação de Imagens - Barzinga

In [1]:
from sklearn.preprocessing import StandardScaler
from time import time
import pandas as pd
import numpy as np

## Carregando Dados

In [2]:
np.random.seed(100)
df = pd.read_csv('dataset.csv', header=None)

In [3]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,7491,7492,7493,7494,7495,7496,7497,7498,7499,7500
0,36494,18504,21074,35980,18761,21074,36237,19018,21331,37522,...,26728,23387,18504,25443,22616,17990,24929,22102,16962,sensacao
1,45489,47031,43947,45489,47031,43947,45489,47031,43947,45489,...,58082,59110,54741,56540,57054,53456,54227,54484,50372,sensacao
2,44461,45489,41120,44461,45489,41634,44204,45232,41634,44461,...,50629,49087,46003,50115,48573,45489,49344,47802,44718,trident
3,49858,52685,51657,50115,52428,51657,48059,50372,50115,41377,...,64250,64250,63736,63993,63736,62708,63479,63222,62194,duo
4,49858,52685,51143,49858,52942,51914,49344,53199,51914,50115,...,64250,63993,62708,63736,62965,61166,63736,62965,60652,duo


In [4]:
df.loc[df[7500] == 'oreo', 7500] = 0
df.loc[df[7500] == 'sensacao', 7500] = 1
df.loc[df[7500] == 'duo', 7500] = 2
df.loc[df[7500] == 'trident', 7500] = 2

## Split de Dados

In [5]:
df_train = df.iloc[:900, :]
df_test = df.iloc[900:, :]

## Features

In [6]:
a = []
for i in range(0, 2500):
    a.append(.2126)
    a.append(.7152)
    a.append(.0722)

a = np.array(a)
type(a)

numpy.ndarray

In [7]:
X_train = (df_train.iloc[:, :-1].values / 255) * a
X_test = (df_test.iloc[:, :-1].values / 255) * a

In [8]:
X_train.shape

(900, 7500)

## Dados

In [9]:
y_train = df_train[7500].values
y_train_onehot = pd.get_dummies(df_train[7500]).values
y_test = df_test[7500].values

## Rede Neural Convolucional

In [10]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Conv2D, MaxPooling2D, Flatten

start = time()

img_rows, img_cols = 50, 150
nb_filters = 16
pool_size = (2, 2)
kernel_size = (4, 4)

X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

model = Sequential()

model.add(Conv2D(nb_filters, kernel_size, input_shape=input_shape))

model.add(Activation('relu'))

model.add(Conv2D(nb_filters, kernel_size))

model.add(Activation('relu'))

model.add(Conv2D(nb_filters, kernel_size))

model.add(Activation('relu'))

model.add(Conv2D(nb_filters, kernel_size))

model.add(Activation('relu'))

model.add(Conv2D(nb_filters, kernel_size))

model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size = pool_size))

model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(128))

model.add(Activation('relu'))

model.add(Dropout(0.3))

model.add(Dense(3))

model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

model.fit(X_train, y_train_onehot, epochs=2)

print ('\nTempo gasto: %s segundos' % str(time() - start))

/home/gabriela.batista/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Epoch 1/2
900/900 [==============================] - 24s 26ms/step - loss: 8.1422 - acc: 0.4811
Epoch 2/2
900/900 [==============================] - 23s 25ms/step - loss: 8.2740 - acc: 0.4867

Tempo gasto: 47.022934675216675 segundos


In [11]:
y_prediction = model.predict_classes(X_test)
print ("\nAcurácia", np.sum(y_prediction == y_test) / float(len(y_test)))


Acurácia 0.44537815126050423


In [12]:
from keras.models import load_model

model.save('barzinga_model.h5')  # creates a HDF5 file 'my_model.h5'
del model  # deletes the existing model

model = load_model('barzinga_model.h5')